
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

     |████████████████████████████████| 175 kB 7.0 MB/s 
     |████████████████████████████████| 356 kB 53.0 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=356d99c1e5ea4fcaa62c589309cca5a132c1028148e5d142efe6931f3db4a711
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491651 sha256=2dc6697e793685b9a450cf1e8f08f3893194e38a493c871f4dbaac56a2b0be05
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built memory-profiler shap
     |████████████████████████████████| 177 kB 8.3 MB/s 


In [2]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 7.1 MB/s 


In [3]:
!pip install BlackBoxAuditing

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394771 sha256=9198df42bf5af11de2b335fdfef6d29ff883a63436d2bda5e41276b91ead8952
  Stored in directory: /root/.cache/pip/wheels/05/9f/ee/541a74be4cf5dad17430e64d3276370ea7b6a834a76cb4215a
Successfully built BlackBoxAuditing


#IMPORTS

In [4]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import BlackBoxAuditing
import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
from aif360.algorithms.inprocessing import PrejudiceRemover, MetaFairClassifier, AdversarialDebiasing
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR,OptimPreproc
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()


import sys
sys.path.append("../")
import os

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


#**************************LOADING DATASET*******************************

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# AdDEb




In [7]:
#initialize Tf
sess = tf.Session() #initialize Tf

In [9]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]
  class Train(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  
  sess.close() # This closse would close the previous iteration  and start a new session for the current iteration
  tf.reset_default_graph()
  sess = tf.Session()
  Classifier = AdversarialDebiasing(privileged_groups = advantagedGroup,
                          unprivileged_groups = disadvantagedGroup,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/AdDeb/AdDeb.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/AdDeb/AdDeb.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 0; iter: 0; batch classifier loss: 0.875159; batch adversarial loss: 0.702591
epoch 1; iter: 0; batch classifier loss: 0.434610; batch adversarial loss: 0.686930
epoch 2; iter: 0; batch classifier loss: 0.490360; batch adversarial loss: 0.718453
epoch 3; iter: 0; batch classifier loss: 0.455390; batch adversarial loss: 0.683238
epoch 4; iter: 0; batch classifier loss: 0.414802; batch adversarial loss: 0.695043
epoch 5; iter: 0; batch classifier loss: 0.365586; batch adversarial loss: 0.679229
epoch 6; iter: 0; batch classifier loss: 0.401969; batch adversarial loss: 0.698208
epoch 7; iter: 0; batch classifier loss: 0.337289; batch adversarial loss: 0.671496
epoch 8; iter: 0; batch classifier loss: 0.328350; batch adversarial loss: 0.664306
epoch 9; iter: 0; batch classifier loss: 0.408738; batch adversarial loss: 0.665287
epoch 10; iter: 0; batch classifier loss: 0.3

divide by zero encountered in double_scalars


Accuracy 0.844139650872818
epoch 0; iter: 0; batch classifier loss: 1.123602; batch adversarial loss: 0.681374
epoch 1; iter: 0; batch classifier loss: 0.573894; batch adversarial loss: 0.681171
epoch 2; iter: 0; batch classifier loss: 0.387984; batch adversarial loss: 0.670809
epoch 3; iter: 0; batch classifier loss: 0.480544; batch adversarial loss: 0.684270
epoch 4; iter: 0; batch classifier loss: 0.349098; batch adversarial loss: 0.677707
epoch 5; iter: 0; batch classifier loss: 0.372485; batch adversarial loss: 0.668643
epoch 6; iter: 0; batch classifier loss: 0.358417; batch adversarial loss: 0.671235
epoch 7; iter: 0; batch classifier loss: 0.339355; batch adversarial loss: 0.662384
epoch 8; iter: 0; batch classifier loss: 0.488846; batch adversarial loss: 0.689212
epoch 9; iter: 0; batch classifier loss: 0.392508; batch adversarial loss: 0.665596
epoch 10; iter: 0; batch classifier loss: 0.353895; batch adversarial loss: 0.652428
epoch 11; iter: 0; batch classifier loss: 0.4511

divide by zero encountered in double_scalars


Accuracy 0.5448877805486284
epoch 0; iter: 0; batch classifier loss: 0.841330; batch adversarial loss: 0.780631
epoch 1; iter: 0; batch classifier loss: 0.624352; batch adversarial loss: 0.739236
epoch 2; iter: 0; batch classifier loss: 0.613192; batch adversarial loss: 0.742323
epoch 3; iter: 0; batch classifier loss: 0.613509; batch adversarial loss: 0.757448
epoch 4; iter: 0; batch classifier loss: 0.758538; batch adversarial loss: 0.766780
epoch 5; iter: 0; batch classifier loss: 0.788825; batch adversarial loss: 0.763560
epoch 6; iter: 0; batch classifier loss: 0.684802; batch adversarial loss: 0.752169
epoch 7; iter: 0; batch classifier loss: 0.810629; batch adversarial loss: 0.740925
epoch 8; iter: 0; batch classifier loss: 0.824270; batch adversarial loss: 0.723377
epoch 9; iter: 0; batch classifier loss: 0.751367; batch adversarial loss: 0.699445
epoch 10; iter: 0; batch classifier loss: 0.761845; batch adversarial loss: 0.674896
epoch 11; iter: 0; batch classifier loss: 0.639

divide by zero encountered in double_scalars


Accuracy 0.8466334164588528
epoch 0; iter: 0; batch classifier loss: 0.840371; batch adversarial loss: 0.666935
epoch 1; iter: 0; batch classifier loss: 0.491549; batch adversarial loss: 0.679689
epoch 2; iter: 0; batch classifier loss: 0.479152; batch adversarial loss: 0.672884
epoch 3; iter: 0; batch classifier loss: 0.360272; batch adversarial loss: 0.671583
epoch 4; iter: 0; batch classifier loss: 0.432266; batch adversarial loss: 0.682191
epoch 5; iter: 0; batch classifier loss: 0.365587; batch adversarial loss: 0.657523
epoch 6; iter: 0; batch classifier loss: 0.340887; batch adversarial loss: 0.666055
epoch 7; iter: 0; batch classifier loss: 0.430630; batch adversarial loss: 0.666848
epoch 8; iter: 0; batch classifier loss: 0.434781; batch adversarial loss: 0.681489
epoch 9; iter: 0; batch classifier loss: 0.431031; batch adversarial loss: 0.682197
epoch 10; iter: 0; batch classifier loss: 0.475822; batch adversarial loss: 0.674872
epoch 11; iter: 0; batch classifier loss: 0.334

divide by zero encountered in double_scalars
